In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import glob
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo
import csv
import json
import re
import os
import psycopg2

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Connecting to Postgres

In [2]:
#Connecting to Postgres
hostname = 'covid19.clksoczr9us2.us-east-1.rds.amazonaws.com'
port_id = 5432
username = 'postgres'
database = 'postgres'
pwd = 'User1234' 

conn = psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd, 
        port = port_id)
print("database opened successfully")

database opened successfully


# Download from Top 10

In [3]:
#Downloading the data from Postgres
df = pd.read_sql('SELECT * FROM top10', conn)
df.head()

,city,province_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,real_values,predicted_values
0,None,Channel Islands,United Kingdom,49.372300,-2.364400,0.0,8.0,2021-04-02,5205505,"[49.3723, -2.3644]",0.0,0.0
1,Loving,Texas,US,31.849476,-103.581857,1.0,1.0,2021-01-30,7182375,"[31.84947612, -103.5818571]",1.0,1.0
2,Harding,New Mexico,US,35.857516,-103.822094,8.0,8.0,2021-01-30,7182375,"[35.85751555, -103.82209379999999]",8.0,8.0
3,Petroleum,Montana,US,47.118246,-108.250888,11.0,10.0,2021-01-30,7182375,"[47.118246299999996, -108.2508876]",11.0,11.0
4,Hinsdale,Colorado,US,37.822474,-107.300434,16.0,16.0,2021-01-30,7182375,"[37.82247444, -107.3004345]",16.0,16.0


In [4]:
# FINAL PROJECT: Import DATABASE
city_data_df = df
city_data_df.head()

,city,province_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,real_values,predicted_values
0,None,Channel Islands,United Kingdom,49.372300,-2.364400,0.0,8.0,2021-04-02,5205505,"[49.3723, -2.3644]",0.0,0.0
1,Loving,Texas,US,31.849476,-103.581857,1.0,1.0,2021-01-30,7182375,"[31.84947612, -103.5818571]",1.0,1.0
2,Harding,New Mexico,US,35.857516,-103.822094,8.0,8.0,2021-01-30,7182375,"[35.85751555, -103.82209379999999]",8.0,8.0
3,Petroleum,Montana,US,47.118246,-108.250888,11.0,10.0,2021-01-30,7182375,"[47.118246299999996, -108.2508876]",11.0,11.0
4,Hinsdale,Colorado,US,37.822474,-107.300434,16.0,16.0,2021-01-30,7182375,"[37.82247444, -107.3004345]",16.0,16.0


In [5]:
city_data_df.dtypes

city                        object
province_state              object
country_region              object
lat                        float64
long                       float64
confirmed_cases            float64
cases_per_day              float64
last_update                 object
people_fully_vaccinated      int64
coordinates                 object
real_values                float64
predicted_values           float64
dtype: object

In [6]:
# FINAL PROJECT: FILTER THE DF USING LOC METHOD
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df
preferred_cities_df.head(10)

,city,province_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,real_values,predicted_values
0,None,Channel Islands,United Kingdom,49.372300,-2.364400,0.0,8.0,2021-04-02,5205505,"[49.3723, -2.3644]",0.0,0.0
1,Loving,Texas,US,31.849476,-103.581857,1.0,1.0,2021-01-30,7182375,"[31.84947612, -103.5818571]",1.0,1.0
2,Harding,New Mexico,US,35.857516,-103.822094,8.0,8.0,2021-01-30,7182375,"[35.85751555, -103.82209379999999]",8.0,8.0
3,Petroleum,Montana,US,47.118246,-108.250888,11.0,10.0,2021-01-30,7182375,"[47.118246299999996, -108.2508876]",11.0,11.0
4,Hinsdale,Colorado,US,37.822474,-107.300434,16.0,16.0,2021-01-30,7182375,"[37.82247444, -107.3004345]",16.0,16.0
5,Skagway,Alaska,US,59.561500,-135.333775,16.0,16.0,2021-01-30,7182375,"[59.56149996, -135.33377480000001]",16.0,16.0
6,Blaine,Nebraska,US,41.913117,-99.976778,19.0,19.0,2021-01-30,7182375,"[41.91311716, -99.97677845]",19.0,19.0
7,None,Montserrat,United Kingdom,16.742498,-62.187366,20.0,6.0,2021-02-25,736037,"[16.742498, -62.187366000000004]",20.0,20.0
8,Wheeler,Oregon,US,44.726982,-120.028143,22.0,21.0,2021-01-30,7182375,"[44.72698162, -120.02814270000002]",22.0,21.9
9,Arthur,Nebraska,US,41.568961,-101.695956,22.0,22.0,2021-01-30,7182375,"[41.56896142, -101.6959559]",22.0,22.0


In [7]:
# FINAL PROJECT: USE BELOW BUT ADD CURRENT AND PROJECTED COVID CASES
# hotel_df = clean_df[["City", "Country", "Current Covid Cases", "Projected Covid Cases", "Current Description", "Lat", "Lng"]].copy()
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
# hotel_df["Hotel Name"] = preferred_cities_df
hotel_df = preferred_cities_df[["country_region", "city", "province_state", "lat", "long", "confirmed_cases", "people_fully_vaccinated", "predicted_values"]].copy()
hotel_df

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0
2,US,Harding,New Mexico,35.857516,-103.822094,8.0,7182375,8.0
3,US,Petroleum,Montana,47.118246,-108.250888,11.0,7182375,11.0
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0
7,United Kingdom,None,Montserrat,16.742498,-62.187366,20.0,736037,20.0
8,US,Wheeler,Oregon,44.726982,-120.028143,22.0,7182375,21.9
9,US,Arthur,Nebraska,41.568961,-101.695956,22.0,7182375,22.0


In [8]:
# Add Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values,Hotel Name
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0,
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0,
2,US,Harding,New Mexico,35.857516,-103.822094,8.0,7182375,8.0,
3,US,Petroleum,Montana,47.118246,-108.250888,11.0,7182375,11.0,
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0,
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0,
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0,
7,United Kingdom,None,Montserrat,16.742498,-62.187366,20.0,736037,20.0,
8,US,Wheeler,Oregon,44.726982,-120.028143,22.0,7182375,21.9,
9,US,Arthur,Nebraska,41.568961,-101.695956,22.0,7182375,22.0,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["lat"]
    lng = row["long"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
print(hotel_df)

   country_region       city   province_state        lat        long  \
0  United Kingdom       None  Channel Islands  49.372300   -2.364400   
1              US     Loving            Texas  31.849476 -103.581857   
2              US    Harding       New Mexico  35.857516 -103.822094   
3              US  Petroleum          Montana  47.118246 -108.250888   
4              US   Hinsdale         Colorado  37.822474 -107.300434   
5              US    Skagway           Alaska  59.561500 -135.333775   
6              US     Blaine         Nebraska  41.913117  -99.976778   
7  United Kingdom       None       Montserrat  16.742498  -62.187366   
8              US    Wheeler           Oregon  44.726982 -120.028143   
9              US     Arthur         Nebraska  41.568961 -101.695956   

   confirmed_cases  people_fully_vaccinated  predicted_values  \
0              0.0                  5205505               0.0   
1              1.0                  7182375               1.0   
2           

In [11]:
# FINAL PROJECT: CREATE OUTPUT FILE
# output_data_file = "ENTER NEW FILE.csv"
# 8a. Create the output File (CSV)
output_data_file = "top_10_hotel_list.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)
hotel_df.head()

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values,Hotel Name
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0,La Sablonnerie
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0,
2,US,Harding,New Mexico,35.857516,-103.822094,8.0,7182375,8.0,
3,US,Petroleum,Montana,47.118246,-108.250888,11.0,7182375,11.0,
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0,


In [12]:
# FINAL PROJECT: UPDATE TEMPLATE TO INCLUDE COVID CASES AND PROJECTED COVID CASES
# info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City Name</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# <dt>Current Weather</dt><dd>{Current Description}</dd>
# <dt>Current Covid Cases</dt><dd>{CURRENT COVID CASES}</dd>
# <dt>Projected Covid Cases</dt><dd>{PROJECTED COVID CASES}</dd>
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# </dl>
# """
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country_region}</dd>
<dt>Confirmed Covid Cases</dt><dd>{confirmed_cases}</dd>
<dt>Projected Covid Cases</dt><dd>{predicted_values}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["lat", "long"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["lat", "long"]]
fig = gmaps.figure(center=(49.372300, -2.364400), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))